# Full End-to-End Example

This tutorial goes through the code of an end-to-end script for rev7E K34, and describes what each part of the code does. The only two parts with human interaction are the frequency calibration GUI and the power calibration GUI. To follow along, go into the pipeline directory and type in:

```
python Rev007EK34_e2e.py
```

In a different window, open the code you just ran (`Rev007EK34_e2e.py`) in your favorite text editor.

Also, for the record, I realize that most of the information in here could reasonably be put into docstrings and comments in the script. However, I think it's a little easier to read in here.

## Importing the rss_ringoccs package

To import the package, just include the base directory and import it:

```python
sys.path.append('..')
import rss_ringoccs as rss
sys.path.remove('..')
```

You need to import the routines to save the four types of data files separately:

```python
sys.path.append('../rss_ringoccs/tools')
from pds3_geo_series import write_geo_series
from pds3_cal_series import write_cal_series
from pds3_dlp_series import write_dlp_series
from pds3_tau_series import write_tau_series
sys.path.remove('../rss_ringoccs/tools')
```

## Finding and Entering an RSR file and Kernels

The set of code in the script specifying the RSR file and kernel locations looks like this:

```python
rsr_file_name = 's10sroe2005123_0740nnnk34rd.1b2'
rsr_file_pds_dir = 'co-s-rss-1-sroc1-v10/cors_0105/sroc1_123/rsr/'
rsr_file_local_dir = "../data/" + rsr_file_pds_dir
kernels_list_file = '../tables/rev007_list_of_kernels.txt'
kernels_dir = '../kernels/'
kernels = '../tables/Rev007_meta_kernel.ker'
rev_number = '007'
occultation_direction = 'Egress'
```

### RSR File

All RSR files are located at https://atmos.nmsu.edu/pdsd/archive/data/. When you go there, you will see a huge list of directories. The radio science data directories are `co-s-rss-1-sroc*-v10/`. The first two lines in here defining the __rsr_file_name__ and __rsr_file_pds_dir__ variables specify where the file is on this website. The __rsr_file_local_dir__ variable tells where to save this file on to your local machine. We recommend you keep the latter as is. In the second-to-last line of this block, we define __rev_number__, which is the orbit number of the occultation. In the text file `../tables/RSSActivities_before_USOfailure_rings_only.txt`, the first three numbers right after "RSS\_" at the beginning of each row are the rev number for that year and day of year (DOY). __occultation_direction__ specifies what direction the occultation is going in (your choices are "Egress", "Ingress", or "Chord").

### Kernels

The __kernels_list_file__ variable is a text file listing all kernels needed for the occultation. A shell script accepts this text file as input, then looks online for these kernels and saves them in a specified directory. In most cases, you can set this to the string `../tables/list_of_kernels.txt`, which covers all occultations. The __kernels_dir__ variable specifies what directory to save this list of kernels in. We recommend you leave the latter as is. The __kernels__ variable is a kernel that encompasses multiple kernel files. In most cases, you can leave this as `../tables/Sa-TC17-V001.ker`

## Downloading RSR file and Kernels

The following block of code uses a pair of shell scripts to download the specified RSR file and kernels:

```python
print("Downloading RSR files...")
os.system('./get_rsr_file.sh %s %s %s ; echo "RSR Complete"' %
          (rsr_file_name, rsr_file_pds_dir, rsr_file_local_dir))
print("Downloading kernels...")
os.system('./get_kernels.sh %s %s ; echo "Kernels Complete"' %
          (kernels_list_file, kernels_dir))
rsr_file = rsr_file_local_dir + rsr_file_name
```

The first `os.system` command uses the shell script to download the RSR file, and the second one uses the shell script to download the kernels. You may also choose to run these shell scripts before running the Python script.

## Specifying Output Files and Directory

All output files and the output directory are specified by the following block of code:

```python
output_directory = '../output/rev7E_K34_e2e_output/'
freq_offset_file = output_directory + 'freq_offset_file.txt'
f_resid_fit_parameters_file = output_directory + 'f_resid_fit_parameters.p'
power_norm_fit_parameters_file = (
    output_directory + 'power_norm_fit_parameters.p'
)
geo_file = 'RSS_2005_123_K34_E_GEO'
cal_file = 'RSS_2005_123_K34_E_CAL'
dlp_file = 'RSS_2005_123_K34_E_DLP'
```

Variable descriptions:
* __output_directory__ - Tells the code the directory in which to save all the output files
* __freq_offset_file__ - The full path of a file in which to save calculated frequency offset
* __f_resid_fit_parameters_file__ - The full path name of a pickle file in which to save the frequency calibration fit parameters. Fit parameters saved are free-space regions in SPM and polynomial fit order
* __power_norm_fit_parameters_file__ - The full path name of a pickle file in which to save the power calibration fit parameters. Fit parameters saved are free-space regions in SPM, knots of the spline fit in SPM, and spline fit order
* __geo_file__ - Name of data file containing geometry information, and its corresponding label file
* __cal_file__ - Name of data file containing calibration information, and its corresponding label file
* __dlp_file__ - Name of data file containing diffraction-limited profile information, and its corresponding label file

Note that the routines that save the last three files will add an underscore, the current date in YYYYMMDD format, and ".TAB" for the data file and ".LBL" for the corresponding label file. __Specifying any of these files is not strictly necessary to running the pipeline.__

## Specifying USO Frequency and Diffraction Correction Parameters

Below the chunk of code specifying output files and directory, you will see:

```python
f_USO = 8427222034.34050 * 3.8
dr_km_desired = 0.25
res_km = 1.0
inversion_range = [87410, 87610]
tau_file = 'RSS_2005_123_K34_E_TAU'
verbose = True
```

Variable descriptions:

* __f_USO__ - USO frequency, which is necessary for the frequency calibration step. The number given in this case (8427222034.34050 \* 3.8) is for Ka band. Drop the factor of 3.8 for X band (8427222034.34050), and multiply the X band value by 3/11 for S band (8427222034.34050 \* (3/11)).
* __dr_km_desired__ - The radial spacing to resample to when transferring to uniformly spaced radius right before diffraction correction.
* __res_km__ - the radial resolution at which the script does diffraction correction, which can't be any lower than twice dr_km_desired.
* __inversion_range__ - Range over which to do diffraction correction. Usually want to make this about [70000, 150000] if you're saving the data files.
* __tau_file__ - Name of the fourth type of data file to save. Adds the same set of information as the geo_file, cal_file, and dlp_file above.
* __verbose__ - Boolean variable to specify whether or not to print out intermediate steps and results. It's set to True right now for the purpose of demonstration.

## Functions to Read and Write Pickle Files

Below the definitions of USO frequency and diffraction correction parameters, you will see four functions defined. The `read_f_resid_fit_parameters` and `write_f_resid_fit_parameters` functions read and write the frequency calibration fit parameters, while the `read_power_norm_fit_parameters` and `write_power_norm_fit_parameters` functions read and write the power calibration fit parameters. These are used later on in the script. All four functions require importing the pickle module (```import pickle```)

## Reading the RSR file and Calculating Occultation Geometry

The following set of lines in the script begin the first steps in the pipeline:

```python
rsr_inst = rss.rsr_reader.RSRReader(rsr_file, verbose=verbose)

rev_info = rss.tools.get_rev_info(rsr_inst, rev_number)

geo_inst = rss.occgeo.Geometry(
    rsr_inst, 'Saturn', 'Cassini', [kernels], verbose=verbose
)
```

The first line defining __rsr_inst__ reads the Radio Science Receiver file specified by the __rsr_file__ variable. The second line retrieves a dictionary with a set of information about the occultation. __This second line is not strictly necessary to run the pipeline, since it is only used when saving the data files.__ The third line defining __geo_inst__ calculates the occultation geometry over the full time range in the RSR file.

## Calculating Frequency Offset and Saving the Results

```python
os.system('[ ! -d ' + output_directory + ' ] && mkdir -p ' + output_directory)

# Calculate frequency offset if no file already there. Otherwise, read in the
#     previously made file
if os.path.exists(freq_offset_file):
    freq_offset_file_vals = np.loadtxt(freq_offset_file)
    f_spm = freq_offset_file_vals[:, 0]
    f_offset = freq_offset_file_vals[:, 1]
else:
    f_spm, f_offset, freq_offset_history = rss.calibration.calc_freq_offset(
        rsr_inst, freq_offset_file=freq_offset_file, verbose=verbose)
```

The first line in the above code creates the output directory if it doesn't exist already. If you're not saving any files, this line is unnecessary.

Once the directory is made, the if-else statement checks if a file exists with previously-calculated frequency offset. If it does, then it loads the file instead of repeating the calculation, and otherwise, it goes through the calculation.

## Frequency Calibration and Saving the Results

```python
# Manually make fit to frequency offset if no file already there. Otherwise,
#     read in the fit parameters from the previously made file
if os.path.exists(f_resid_fit_parameters_file):
    k_f_resid, spm_include = read_f_resid_fit_parameters(
        f_resid_fit_parameters_file
    )
    fit_inst = rss.calibration.FreqOffsetFit(
        rsr_inst, geo_inst, f_spm, f_offset, f_USO, poly_order=k_f_resid,
        spm_include=spm_include, USE_GUI=False, verbose=verbose
    )
else:
    fit_inst = rss.calibration.FreqOffsetFit(
        rsr_inst, geo_inst, f_spm, f_offset, f_USO, verbose=verbose
    )
    write_f_resid_fit_parameters(fit_inst, f_resid_fit_parameters_file)

spm_vals, IQ_c = fit_inst.get_IQ_c()
```

Similar to the frequency offset calculation, the if-else statement checks if a pickle file of fit parameters exists. If it does, then it uses those without calling up the GUI, and if it doesn't, then it calls up the GUI and then saves the resulting fit parameters. The line after the if-else statement uses the fit to frequency offset in __fit_inst__ to frequency correct the complex-valued data at raw resolution.

## Power Calibration and Saving the Results

```python
norm_inst = rss.calibration.Normalization(
    spm_vals, IQ_c, geo_inst, rsr_inst, verbose=verbose
)

# Manually make spline fit to unnormalized power if no file already there.
#     Otherwise, read in the fit parameters from the previously made file
if os.path.exists(power_norm_fit_parameters_file):
    k_power_norm, freespace_spm, knots_spm = read_power_norm_fit_parameters(
        power_norm_fit_parameters_file
    )
    spm_power_fit, power_spline_fit = norm_inst.get_spline_fit(
        freespace_spm=freespace_spm, knots_spm=knots_spm,
        spline_order=k_power_norm, USE_GUI=False, verbose=verbose
    )
else:
    spm_power_fit, power_spline_fit = norm_inst.get_spline_fit(verbose=verbose)
    write_power_norm_fit_parameters(
        norm_inst, power_norm_fit_parameters_file
    )
```

The top line defines an instance of the Normalization class, __norm_inst__, which is used in the if-else statement to make a spline fit to free-space power for normalization. Once again similar to the preceeding steps, the if-else statement checks if a pickle file with spline fit parameters exists. If it does, then it uses those parameters without calling the GUI, and if it doesn't, then it uses the GUI and saves the resulting fit parameters.

## Diffraction Correction

```python
cal_inst = rss.calibration.Calibration(
    fit_inst, norm_inst, geo_inst, verbose=verbose
)

dlp_inst = rss.calibration.NormDiff(
    rsr_inst, dr_km_desired, geo_inst, cal_inst, verbose=verbose
)

tau_inst = rss.diffcorr.DiffractionCorrection(
    dlp_inst, res_km, rng=inversion_range, verbose=verbose
)
```

Variable descriptions:

* __cal_inst__ - Instance of the Calibration class, which basically just lumps together the information from the two calibration steps.
* __dlp_inst__ - Instance of the NormDiff class, which applies all the calibration information in __cal_inst__ to the raw data to produce a normalized diffraction pattern. After the calibration steps are applied, it resamples to uniformly spaced radius using the __resample_IQ__ function found in the rss_ringoccs/calibration/ directory. As a note "DLP" stands for "Diffraction-Limited Profile".
* __tau_inst__ - Instance of the DiffractionCorrection class, which performs diffraction correction at a specified resolution and radial range using __dlp_inst__ as input.

## Writing Data Files

```python
write_geo_series(rev_info, geo_inst, geo_file, output_directory, occultation_direction)
write_cal_series(rev_info, cal_inst, cal_file, output_directory, occultation_direction)
write_dlp_series(rev_info, dlp_inst, dlp_file, output_directory, occultation_direction)
write_tau_series(rev_info, tau_inst, tau_file, output_directory, occultation_direction)
```

Each line writes out a data file and a corresponding label file. __This step is not strictly necessary to running the pipeline__.

## Adapting the Script to Other Occultations

To run a different occultation, these are the variables you would need to worry about changing:

* __rsr_file_name__ - Name of RSR file
* __rsr_file_pds_dir__ - Directory of the RSR file on the online PDS node
* __kernels_list_file__ - If you change this to the file specified in the "Finding and Entering an RSR file and Kernels" section, that should work for more or less all occultations
* __kernels__ - If you change this to the file specified in the "Finding and Entering an RSR file and Kernels" section, that should work for more or less all occultations
* __rev_number__ - Orbit number of the occultation.
* __output_directory__ - Whatever you change it to, we recommend you still put it in the "output" directory in rss_ringoccs
* __geo_file, cal_file, dlp_file, tau_file__ - We recommend you follow the same format as what's in this example script: "RSS_YYYY_DOY_bnn_D_FFF", where "b" means band (S, X, or K), "nn" means DSN station number (34, e.g.), "D" means occultation direction (I, E, or C), and "FFF" means file type (GEO, CAL, or DLP).
* __f_USO__ - Just add or remove the multiplying factor to X-band USO frequency as needed. So delete the " * 3.8" for X band, keep it as is for K band, and replace the "3.8" with "3/11" for S band.
* __dr_km_desired__ - Just make it whatever radial spacing you want.
* __res_km__ - Just make it whatever processing resolution you want.
* __inversion_range__ - Just make it whatever radial range you want to do diffraction correction on.

## Using the Data Files for Quick-Look

To see how to use the data files to redo diffraction correction without redoing the calibration steps, see the script __Rev007EK34_quick_look.py__. You will need to replace the `geo_file`, `cal_file`, and `dlp_file` variables with the names of your output from the __Rev007EK34_e2e.py__ script outlined in the rest of this tutorial. To see different ring features at different processing resolutions, play with the `res_km` and `inversion_range` variables. The only two things you need from the `rss_ringoccs` package when using the data files are `rss_ringoccs.tools.ExtractCSVData` and `rss_ringoccs.diffcorr.DiffractionCorrection`.